<a href="https://colab.research.google.com/github/Kishan-Ved/MLDeepLab/blob/main/forward_propogation_neural_networks_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Neural Network from scratch**
This colab contains forward propogation in a neural network written from scratch in python.

The aim is to create a neural network to predict whether roasting of coffee beans is done properly based its on temperature and duration.

The logic is the same as logistic regression.

##**Generating coffee roasting data**

In [24]:
import numpy as np

def load_coffee_data():
    # Creates a coffee roasting data set.
    # roasting duration: 12-15 minutes is best
    # temperature range: 175-260C is best

    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [25]:
X,Y = load_coffee_data()

In [26]:
print(X)

[[185.31763812  12.69396457]
 [259.92047498  11.86766377]
 [231.01357101  14.41424211]
 [175.3666449   11.72058651]
 [187.12086467  14.12973206]
 [225.90586448  12.10024905]
 [208.40515676  14.17718919]
 [207.07593089  14.0327376 ]
 [280.60385359  14.23225929]
 [202.86935247  12.24901028]
 [196.70468985  13.54426389]
 [270.31327028  14.60225577]
 [192.94979108  15.19686759]
 [213.57283453  14.27503537]
 [164.47298664  11.91817423]
 [177.25750542  15.03779869]
 [241.7745473   14.89694529]
 [236.99889634  13.12616959]
 [219.73805621  13.87377407]
 [266.38592796  13.25274466]
 [270.45241485  13.95486775]
 [261.96307698  13.49222422]
 [243.4899478   12.8561015 ]
 [220.58184803  12.36489356]
 [163.59498627  11.65441652]
 [244.76317931  13.32572248]
 [271.19410986  14.84073282]
 [201.98784315  15.39471508]
 [229.9283715   14.56353326]
 [204.97123839  12.28467965]
 [173.18989704  12.2248249 ]
 [231.51374483  11.95053142]
 [152.68795109  14.83198786]
 [163.42050092  13.30233814]
 [215.94730737

In [27]:
print(Y)

[[1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.

##**Normalizing the data**

In [45]:
def Normalization_(X): # Same as tensorflow.keras.layers.Normalization(axis=-1)
  n_samples = X.shape[0]
  n_features = X.shape[1]
  mean_X = np.zeros(n_features)
  std_X = np.zeros(n_features)

  for i in range(n_features):
    for j in range(n_samples):
      mean_X[i] += X[j][i]
    mean_X[i]/=n_samples

  for i in range(n_features):
    for j in range(n_samples):
      std_X[i]+=(X[j][i]-mean_X[i])**2
    std_X[i]/=n_samples
    std_X[i] = std_X[i]**0.5


  for i in range(n_features):
    for j in range(n_samples):
      if(std_X[i]==0): std_X[i]=0.000001
      X[j][i] = (X[j][i]-mean_X[i])/std_X[i]

  return X


The normalized data looks like this:

In [29]:
Xn = Normalization_(X)
print(Xn)

[[-0.83455487 -0.65287939]
 [ 1.03230561 -1.38514475]
 [ 0.3089396   0.87162554]
 [-1.08356841 -1.51548419]
 [-0.78943095  0.61949357]
 [ 0.18112441 -1.17902835]
 [-0.25681309  0.66154994]
 [-0.29007562  0.53353736]
 [ 1.54988623  0.7103529 ]
 [-0.39534099 -1.04719659]
 [-0.54960543  0.10065339]
 [ 1.29237479  1.0382427 ]
 [-0.64356797  1.56518596]
 [-0.12749718  0.74826093]
 [-1.35617118 -1.34038253]
 [-1.03625154  1.42421957]
 [ 0.57822214  1.2993957 ]
 [ 0.4587163  -0.26986057]
 [ 0.02678125  0.39266434]
 [ 1.19409703 -0.1576901 ]
 [ 1.29585675  0.46452933]
 [ 1.08341966  0.05453598]
 [ 0.6211483  -0.50919412]
 [ 0.04789629 -0.94450116]
 [-1.37814225 -1.57412384]
 [ 0.65300963 -0.09301737]
 [ 1.31441691  1.24958036]
 [-0.41739987  1.74051783]
 [ 0.28178358  1.00392692]
 [-0.34274342 -1.0155865 ]
 [-1.13803932 -1.0686296 ]
 [ 0.32145594 -1.31170768]
 [-1.65107977  1.2418306 ]
 [-1.38250857 -0.11374051]
 [-0.0680784   0.48776708]
 [-0.01566196  1.61171572]
 [ 0.81667554  0.32539483]
 

##**Activation function - the Sigmoid function**

In [30]:
def sigmoid(z):
    z = np.clip(z, -500, 500 ) # protect against overflow
    g = 1.0/(1.0+np.exp(-z))
    return g

##**Dense function to create a new layer in our neural network**

In [31]:
def dense_(a_in, W, b, g): # This function creates a layer in our neural network
  # W is a matrix where columns correspond to arrays w to be used

  n = W.shape[1] # This is the number of neurons in the layer (as number of w arrays = number of neurons)
  a_out = np.zeros(n) # Output/activation of this layer

  for i in range(n):
    w = W[:,i] # Extract columns from W
    z = np.dot(w,a_in) + b[i]
    a_out[i] = g(z)
  return a_out

##**Sequential function to connect layers together one after another**

In [32]:
def sequential_(X, W1, b1, W2, b2):
  a1 = dense_(X, W1, b1, sigmoid)
  a2 = dense_(a1, W2, b2, sigmoid)
  ans = a2
  return ans

##**Inferencing our neural network**

Inferencing a neural network is to test the predictions against pre trained values of parameters.

In [33]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] ) # 2 X 3
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] ) # 3 X 1
b2_tmp = np.array( [15.41] )

##**Predict function to predict values based on our neural network**

In [37]:
def predict_(X,W1_tmp,b1_tmp,W2_tmp,b2_tmp):
  Xn = Normalization_(X)
  n = Xn.shape[0]
  pred = np.zeros(n)
  for i in range(n):
    pred[i] = sequential_(Xn[i],W1_tmp,b1_tmp,W2_tmp,b2_tmp)

  yhat = (pred>=0.5).astype(int)
  return yhat

##**Testing our neural network**

In [46]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
predictions = predict_(X_tst, W1_tmp, b1_tmp, W2_tmp, b2_tmp)
print(predictions)

[1 0]
